In [2]:
import logging
import pandas as pd
import numpy as np
import os
import re
import pandas_profiling as pdp

from typing import List



_logger = logging.getLogger(__name__)

In [10]:
drop_studiekeuze_cols = [
    "IndicatieIntensiefProgramma",
    "SelectieEisenIntensiefProgramma",
    "VerhoogdCollegegeldIntensiefProgramma",
    "BezoekadresStraat",
    "BezoekadresNummer",
    "BezoekadresPostcode",
    "Bekostiging",
    "AssociateDegree",
    "JointDegree",
    "BSAopmerking",
    "Studielast",
    "AlgemeenOordeel",
    "SourceURL",  # soms URL gevuld --> mogelijk nieuwe bronnen ?
    "TitelHodex",
    "Afstandonderwijs",
    "Avondonderwijs",
    "HeeftBindendStudieAdvies",
    "HeeftExtraInstroommoment",
    "ExtraStudiekosten",
    "LangeBeschrijvingOpleiding",
    "Opleidingswebsite",
    "PercentageStudentenNaarBuitenland",
    "PercentageStudentenUitBuitenland",  # mogelijk interessant voor techstudies ?
    "PercentageDocentenUitBuitenland",
    "PercentageDocentenNaarBuitenland",
    "PercentageAfstandsonderwijs",
    "PercentageAvondonderwijs",
    "WerkgroepGrootte",
    "IsKoepelOpleiding",
    "StudiekeuzecheckNaam",
    "StudiekeuzecheckBeleidVoor1Mei",
    "StudiekeuzecheckBeleidNa1Mei",
    "StudiekeuzecheckVerplicht",
    "StudiekeuzecheckUrl",
    "StudiekeuzecheckOmschrijving",
    "HeeftHonoursProgramma",
    "InstroomVoltijd",
    "InstroomDeeltijd",
    "InstroomDuaal",
    "InstroomTotaal",
    "InstroomMan",
    "InstroomVrouw",
    "AantalStudentenVoltijd",
    "AantalStudentenDuaal",
    "AantalStudentenTotaal",
    "PercentageMannelijkeStudenten",
    "PercentageVrouwelijkeStudenten",
    "AlgemeenOordeel01Wx",
    "nAlgemeenOordeel01",
    "AlgemeenOordeel01aWPrc",
    "AlgemeenOordeel01bWPrc",
    "AlgemeenOordeel01cWPrc",
    "AlgemeenOordeel01dWPrc",
    "AlgemeenOordeel01eWPrc",
    "ContacttijdEersteJaarVoltijd",
    "ContacttijdEerstejaarsVoltijdAantal",
    "WettelijkeVooropleidingseisenVwo",
    "OpmerkingenBijWettelijkeVooropleidingseisenVwo",
    "WettelijkeAanvullendeEisenVwo",
    "WettelijkeVooropleidingseisenHavo",
    "OpmerkingenBijWettelijkeVooropleidingseisenHavo",
    "WettelijkeAanvullendeEisenHavo",
    "ToelatingsEisenMbo",
    "ToelatingseisenHboWoBachelor",
    "maxjaar1CIJFHO",
    "AfstandsonderwijsPrc",
    "AvondonderwijsPrc",
    "NSEEnqueteJaar",
]


In [11]:
def prepare_sdb_opleidingen_file(path, file: str,
                                 data_quality_report=None) -> pd.DataFrame:
    """
    Read in the Studiekeuze excel Opleidingen Sheet
    TODO: needs to point to the API calls
    :param path: path to SDB excel
    :param file: name of SDB excel file
    :param data_quality_report: generates and stores a Pandas profiling report
    :return: formatted SDB file
    """
    df = pd.read_excel(os.path.join(str(path), file), sheet_name="Opleidingen")

    if data_quality_report:

        logging.info(
            f"Generating Data quality report. Storing : {PATH_TO_DATA_QUALITY_REPORT}"
        )
        sdb_profile_report = pdp.ProfileReport(sdb_all)
        sdb_profile_report.to_file(
            PATH_TO_DATA_QUALITY_REPORT + "sdb_data_quality_report.html"
        )

    logging.info(f"dropping columns: \n {drop_studiekeuze_cols}")
    for col in drop_studiekeuze_cols:
        df = df.drop(col, axis=1)
    df.columns = df.columns.str.lower()
    # check with Tekkieworden
    # logging.info(f"putting filter on:{df.actieveopleiding.name} == 1.0")
    # df = df[df.actieveopleiding == 1.0]

    df.columns = df.columns + "_sdb"

    str_cols = ["brinnummer_sdb", "opleidingscode_sdb", "opleiding_sk123id_sdb"]
    for col in str_cols:
        df[col] = df[col].astype(str)
    df["soortopleiding_sdb"] = df["soortopleiding_sdb"].str.lower()  # like in DUO file
    df["soortho_sdb"] = df["soortho_sdb"].apply(lambda x: x.lower())  # like in DUO file

    # cat opleidingsvormen into one column
    df["opleidingsvorm_sdb"] = np.where(
        df.voltijd_sdb == 1,
        "voltijd onderwijs",
        (np.where(df.deeltijd_sdb == 1, "deeltijd onderwijs", "duaal onderwijs")),
    )
    for col in ["voltijd_sdb", "deeltijd_sdb", "duaal_sdb"]:
        df = df.drop(col, axis=1)

    logging.info(f"Studiekeuze opleidingen frame shape: {df.shape}")
    # logging.info(f'Extract \n: {df.head(5)}')

    return df

In [12]:
prepare_sdb_opleidingen_file(path="../datasets/raw/", file="studiekeuze123_all_20200417.xlsx")

,opleiding_sk123id_sdb,brinnummer_sdb,brinvolgnummer_sdb,opleidingscode_sdb,opleidingscodesamen_sdb,naamopleiding_sdb,soortopleiding_sdb,soortho_sdb,sectorcroho_sdb,startdatum_sdb,...,kortebeschrijvingopleiding_sdb,aantalstudentendeeltijd_sdb,kortebeschrijvingopleidingengels_sdb,langebeschrijvingopleidingengels_sdb,eerstejaarsaantal_sdb,eerstejaarsvrouwprc_sdb,studentenaantal_sdb,vrouwprc_sdb,naamopleidingengels_sdb,opleidingsvorm_sdb
0,19851,24HR,11,34671,34671,Informatie en Communicatie Technologie,bachelor,hbo,Techniek,2015-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Information & Communication Technology,deeltijd onderwijs
1,20255,15BK,0,80142,80142,Pedagogisch Educatief Professional,associate degree,hbo,Gedrag en Maatschappij,2018-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pedagogisch Educatief Professional,deeltijd onderwijs
2,16845,15BK,0,70139,70139,Educational Leadership,master,hbo,Onderwijs,2010-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Educational Leadership,deeltijd onderwijs
3,20327,24FW,1,80080,80080,Ondernemen,associate degree,hbo,Economie,2019-08-21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entrepreneurship,voltijd onderwijs
4,20122,24FW,1,39205,39205,HBO - Rechten,bachelor,hbo,Recht,2017-12-29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Law,voltijd onderwijs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3053,20276,24GL,2,39205,39205,HBO - Rechten,bachelor,hbo,Recht,2015-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Law,deeltijd onderwijs
3054,20300,30YF,5,30040,30040,Toegepaste Fotografie en Beeldcommunicatie,bachelor,hbo,Taal en Cultuur,2019-08-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Applied Photography,voltijd onderwijs
3055,20301,30YF,5,35526,35526,Film en Audiovisuele Media,bachelor,hbo,Taal en Cultuur,2019-08-01,...,De opleiding Film en Audiovisuele Media vormt ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Film and Audio-visual Media,voltijd onderwijs
3056,20332,30WG,2,80161,80161,ICT-Engineer,associate degree,hbo,Techniek,2019-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ICT-Engineer,deeltijd onderwijs
